In [1]:
import findspark
import os

findspark.init()

import pandas as pd 
import numpy as np
from sklearn import preprocessing
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, mean
from pyspark.sql.types import *

sc = pyspark.SparkContext()

In [2]:
path_train = '/Users/tiegsti.solomon/Downloads'
path_test = '/Users/tiegsti.solomon/Downloads'

# we are taking the training outputs generated by the spark cluster and using the 
# test data provided by outbrain to validate + compare labels

rdd_train = sc.textFile(path_test+"/content_reco_features_.csv")
rdd_test = sc.textFile(path_train+"/content_reco_features_test2.csv")

print "No. of Lines: ", rdd_train.count()
print "No. of Partitions: " , rdd_train.getNumPartitions() # 2 partitions

No. of Lines:  576853
No. of Partitions:  2


In [3]:
# remove header train
rdd_train_head = rdd_train.filter(lambda l: "uuid" in l)
rdd_train = rdd_train.subtract(rdd_train_head)

# remove header test
rdd_test_head = rdd_test.filter(lambda l: "uuid" in l)
rdd_test = rdd_test.subtract(rdd_test_head)

In [4]:
rdd_train = rdd_train.map(lambda a: a.split(","))
rdd_test = rdd_test.map(lambda a: a.split(","))

In [5]:
from pyspark.sql.functions import udf, col

sqlContext = SQLContext(sc)
train_df = sqlContext.createDataFrame(rdd_train) # schema err
test_df = sqlContext.createDataFrame(rdd_test)

df_train = train_df.drop('_2','_4','_5','_14','_15','_16')
df_train = df_train.drop('_6','_7','_8','_10','_11','_12')
df_train = df_train.drop('_13')

In [6]:
#train_feat = train_df
# test_feat = test_df.columns

train_dfp = df_train.toPandas()
test_dfp = test_df.toPandas()

In [7]:
from sklearn.preprocessing import LabelEncoder

le = preprocessing.LabelEncoder()

le.fit(train_dfp['_1'].values)
train_dfp['_1'] = le.transform(train_dfp['_1'])

le.fit(test_dfp['_1'].values)
test_dfp['_1'] = le.transform(test_dfp['_1'])
train_dfp.head(10)

,_1,_3,_9
0,23301,140874,0
1,101228,162985,0
2,15073,124343,1
3,48764,319252,1
4,22930,152335,0
5,78781,162984,1
6,3692,134758,0
7,61132,60620,0
8,19784,214724,0
9,97308,232313,1


In [9]:
train_sql = sqlContext.createDataFrame(train_dfp)
train_rdd = train_sql.rdd

training_RDD, validation_RDD = train_rdd.randomSplit([8, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
#test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

#test_RDD.take(5)

In [10]:
# import pandas as pd 
# import numpy as np
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()

# train_targets = train_dfp['_9']

In [11]:
from pyspark.mllib.recommendation import ALS
import math

best_rank = -1
seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02
min_error = float('inf')
error_plot = []
rank_plot = []

for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).reduce(lambda x, y: x + y)/rates_and_preds.count()
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank
    # plot errors
    error_plot.append(errors)
    rank_plot.append(rank)
    

print 'Optimum Rank %s' % best_rank
print 'MSE: %s' % (MSE)
print 'RMSE: %s' % (error)

For rank 4 the RMSE is 0.436757509363
For rank 8 the RMSE is 0.425385519002
For rank 12 the RMSE is 0.421103738817
Optimum Rank 12
MSE: 0.177328358845
RMSE: 0.421103738817


In [12]:
predictions.take(3)

[((74884, 145931), 0.0),
 ((32676, 301183), 0.12176356568537278),
 ((103184, 208320), 0.0)]

In [13]:
rates_and_preds.take(3)

[((59479, 101747), (0.0, 0.23035372545354438)),
 ((92119, 12651), (0.0, 0.14300150462219194)),
 ((1043, 204447), (0.0, -0.1523240376820758))]

In [ ]:
def dcg(predicted_rel, rank):
    predicted_rel = np.asarray(predicted_rel)[:rank]
    n_relevances = len(predicted_rel)
    if n_relevances == 0:
        return float(0)

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(predicted_rel / discounts)


def ndcg(predicted_rel, rank):
    high_sc_dcg = dcg(sorted(predicted_rel, reverse=True), rank)
    if high_sc_dcg == 0:
        return float(0)

    return dcg(predicted_rel, rank) / high_sc_dcg

In [ ]:
# lets test our final ranking results

# True Relevance DCG
dcg([0, 0, 1, 0, 1, 2, 1, 1, 0, 0], rank=12)

# Predicted Relevance DCG
dcg([0, 0, 1, 0, 3, 1, 1, 3, 0, 1], rank=12)

In [ ]:

# Accurate DCG
ndcg([0, 0, 1, 0, 1, 2, 1, 1, 0, 0], rank=12)


# Predicted DCG
ndcg([0, 0, 1, 0, 3, 1, 1, 3, 0, 1], rank=12)